<a href="https://colab.research.google.com/github/LondonNode/Pearl-tutorials/blob/main/5_Settings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pearll

# Introduction

Now we've gone through the main modules you'll be interacting with in Pearl, we can take a look at how you can configure them when initializing an agent via the settings objects. These allow us to group together parameters for each module.

The settings are implemented as [dataclasses](https://docs.python.org/3/library/dataclasses.html), which are useful because they tell you exactly what should go in them and are formatted nicely when printed.

Here is a [link](https://github.com/LondonNode/Pearl/blob/main/pearll/settings.py) to the file with all the settings defined. These have docstrings that should provide insight into what each one does. It was important to highlight this in its own tutorial since this is how agent behaviour is controlled!

# Settings

The `Settings` dataclass acts as the base for all other settings objects.

In [2]:
def f(a: int):
  return a

def g(a: int, b: str):
  return b

In [3]:
from pearll.settings import Settings

from dataclasses import dataclass
from typing import Optional


@dataclass
class YourSettings(Settings):
  a: int
  b: Optional[str] = None

setting = YourSettings(a=3)
print(f"Setting after initialization: {setting}")
# filter_none() removes any None parameters and converts to a dictionary so
# the parameters can be easily passed to their modules via **setting.
setting = setting.filter_none()
print(f"Setting after filtering: {setting}")

# for example now suppose the functions f() and g() belong to the same
# module hierarchy with some shared parameters. We want to define the setting
# for f() though:
print(f"a = {f(**setting)}")

Setting after initialization: YourSettings(a=3, b=None)
Setting after filtering: {'a': 3}
a = 3


# Extending Implemented Settings

If you've made your own instance of a module within Pearl (e.g. a new type of buffer), then you might have new parameters that you want to control at the agent level. This can be done by simply extending the existing settings object for that module. We'll continue with the example of a new buffer here, creating a settings object for the `HERBuffer` also implemented in Pearl.

In [4]:
# This is the actual BufferSettings implemented in Pearl, it only defines the
# buffer size since the other initialization parameters are also passed to the
# high level agent interface directly already (e.g. the gym environment).
 
from pearll.settings import Settings

from dataclasses import dataclass


@dataclass
class BufferSettings(Settings):
  """
  Settings for buffers

  :param buffer_size: max number of transitions to store at once in each environment
  """

  buffer_size: int = int(1e6)

In [5]:
from pearll.settings import BufferSettings
# HERBuffer is derived from BaseBuffer
from pearll.buffers import HERBuffer

from dataclasses import dataclass
import gym


env = gym.make("CartPole-v1")

# HERSettings is derived from BufferSettings
@dataclass
class HERSettings(BufferSettings):
  goal_selection_strategy: str = "future"
  n_sampled_goal: int = 4

print(HERSettings())

settings = HERSettings().filter_none()
buffer = HERBuffer(env=env, **settings)

HERSettings(buffer_size=1000000, goal_selection_strategy='future', n_sampled_goal=4)
